In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d niharika41298/withwithout-mask

In [ ]:
!mkdir data
!unzip withwithout-mask.zip -d data

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
# set train and test path, the dataset already has been divied to train and test
train_path = '/content/data/maskdata/maskdata/train'
test_path =  '/content/data/maskdata/maskdata/test'

In [ ]:
# define train and test batches using vgg16 preprocessing
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['with_mask', 'without_mask'], batch_size=10
)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), classes=['with_mask', 'without_mask'], batch_size=10
)

In [ ]:
# call 10 images with their labels
imgs, labels = next(train_batches)

In [ ]:
# this is optional
def plot_img(images_array):
  fig, axes = plt.subplots(1, 10, figsize=(20,20))
  # axes = axes.flatten()
  for img, ax in zip(images_array, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

plot_img(imgs)
print(labels) #one hot encoding

In [ ]:
# download vgg16 model
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
# vgg16 is not sequential model. so it needs to change to sequential and left one layer because it has 1000 outputs
model = Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(224, 
                                                              224,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1)
])
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

In [ ]:
# freeze all downloaded layers
for layer in model.layers:
  layer.trainable = False 

In [ ]:
# add our won output layer
model.add(Dense(2, activation='softmax'))

In [ ]:
# compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit model
history = model.fit(train_batches, epochs=15, verbose=1)

In [ ]:
#  plot 10 images of test batches
test_imgs, test_labels = next(test_batches)
plot_img(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
# predict
predictions = model.predict(test_batches)
predictions

In [ ]:
model.evaluate(test_batches)

In [ ]:
np.round(predictions) #float to decimal

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# plot confussion matrix
cm_plot_labels = ['with_mask', 'without_mask']
plot_confusion_matrix(cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
test_batches.class_indices

In [ ]:
# import images
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predictions = model.predict(images)
  predicted_digit = np.argmax(predictions, axis=-1)
  class_names = ['with_mask', 'without_mask']
  label = class_names[predicted_digit[0]]
  accuracy = 100*np.max(predictions)

  plt.imshow(img, cmap=plt.cm.gray)
  plt.xticks([])
  plt.yticks([])
  plt.title('{} ({:2.0f}%)'.format(label, accuracy))